In [2]:
# Mount GDrive (for exports)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
import glob

Mounted at /content/drive


In [3]:
folder_path = '/content/drive/MyDrive/POs to Flatten (2024-10-24)/*.csv'
csv_files = glob.glob(folder_path)

In [4]:
import gspread
from google.colab import auth
from gspread_dataframe import set_with_dataframe
from google.auth import default

# Step 1: Authenticate the Colab instance with your Google account
auth.authenticate_user()

# Step 2: Create the Google Sheets client
creds, _ = default()  # Automatically gets the OAuth credentials
gc = gspread.authorize(creds)


In [5]:
# Function to drop unwanted columns
def drop_unwanted_columns(df, cols_to_drop):
    return df.drop(columns=[col for col in cols_to_drop if col in df.columns])

# Function to identify columns after 'Build' or 'Cut' for pivoting
def get_cols_to_pivot(df):
    for target_col in ['Length', 'Build', 'Cut']:
        if target_col in df.columns:
            col_index = df.columns.get_loc(target_col)
            return df.columns[col_index + 1:].tolist()  # Columns after the target
    return []

# Function to split the 'Size' column into separate columns
def split_size_column(df):
    size_split = df['Size'].str.split(' ', expand=True)
    size_split.columns = [f'size_{i+1}' for i in range(size_split.shape[1])]
    df = df.drop('Size', axis=1)
    return pd.concat([df, size_split], axis=1)

# Function to validate quantities and identify problematic files
def validate_quantities(df_raw, df, file_path, file_number):
    original_qty = df_raw['Order Qty'].sum()
    final_qty = df['Qty'].sum()
    if original_qty != final_qty:
        print(f'🔴Problematic File (Original Quantity <> Final Quantity): {file_path}')
        print(f"Original Quantity: {original_qty}")
        print(f"Final Quantity: {final_qty}")
        print(f"Adding to problematic files.")
        problematic_files.append(file_number)
    return original_qty, final_qty

# Function to write DataFrame to a new sheet in Google Sheets
def write_to_google_sheet(df, sheet_name):
    try:
        # Check if the sheet exists, and if so, delete it
        worksheet = spreadsheet.worksheet(sheet_name)
        spreadsheet.del_worksheet(worksheet)
    except gspread.exceptions.WorksheetNotFound:
        pass  # Sheet does not exist, so we can create it directly

    # Create a new sheet and write the DataFrame
    worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=1000, cols=50)
    set_with_dataframe(worksheet, df)

In [6]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default

# Authenticate and create the gspread client
creds, _ = default()
gc = gspread.authorize(creds)

# Open the Google Sheet by URL
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1DyWhdfr_lIvV8jnCJWCu3FsphZTfz4Y1jNvu-TpCId0/edit#gid=897566704"
spreadsheet = gc.open_by_url(spreadsheet_url)

# Define file lists and counters
files_processed = []
problematic_files = ['00078', '00079']
already_ats_files = ['00078']
a = 0

# Main processing loop
cols_to_drop = [
    'Order Qty', 'Production Cost (In Supp Curr)', 'Supplier Currency',
    'Total Cost', 'ACTUAL EX FACTORY', 'Production Costs (In Supp Curr)'
]

for file_path in csv_files:
    file_name = file_path.split('/')[-1]
    file_number = file_name.split('-')[0]

    files_processed.append(file_name)

    if file_number in problematic_files:
        continue

    # Step 1: Read the CSV file
    df_raw = pd.read_csv(file_path, skiprows=1)
    print(f'〰️Processing file: {file_name}')

    # Step 2: Drop unwanted columns
    df = drop_unwanted_columns(df_raw, cols_to_drop)

    # Step 3: Identify columns to pivot
    cols_to_pivot = get_cols_to_pivot(df)
    cols_to_not_pivot = [col for col in df.columns if col not in cols_to_pivot]

    # Step 4: Pivot the DataFrame
    df = pd.melt(
        df,
        id_vars=cols_to_not_pivot,
        value_vars=cols_to_pivot,
        var_name='Size',
        value_name='Qty'
    )

    # Step 5: Split the 'Size' column
    df = split_size_column(df)
    df['PO Number'] = file_number
    df['File Name'] = file_name

    # Step 6: Display the transformed DataFrame
    # display(df)

    # Step 7: Validate quantities
    original_qty, final_qty = validate_quantities(df_raw, df, file_path, file_number)

    # Step 8: Write to Google Sheet
    write_to_google_sheet(df, file_name)  # Using file_name as sheet name

# Print final problematic files list
print('Problematic Files:', problematic_files)
print('Files Processed:', files_processed)


〰️Processing file: 00122-Tal Global Alliances Limited - Supplier PO - Data.csv
〰️Processing file: 00120-HKS - Supplier PO - Data.csv
〰️Processing file: 00123-Valerius Textils SA - Supplier PO - Data.csv
〰️Processing file: 00114-Star Garments - Supplier PO - Data.csv
〰️Processing file: 00112-Saitex International Dong Nai - Supplier PO - Data.csv
🔴Problematic File (Original Quantity <> Final Quantity): /content/drive/MyDrive/POs to Flatten (2024-10-24)/00112-Saitex International Dong Nai - Supplier PO - Data.csv
Original Quantity: 500
Final Quantity: 0
Adding to problematic files.
〰️Processing file: 00121-Tal Global Alliances Limited - Supplier PO - Data.csv
〰️Processing file: 00119-HKS - Supplier PO - Data.csv
〰️Processing file: 00115-HKS - Supplier PO - Data.csv
〰️Processing file: 00117-Star Garments - Supplier PO - Data.csv
〰️Processing file: 00113-Lamosa Jeans - Supplier PO - Data.csv
〰️Processing file: 00111-Saitex International Dong Nai - Supplier PO - Data.csv
🔴Problematic File (O

APIError: {'code': 429, 'message': "Quota exceeded for quota metric 'Write requests' and limit 'Write requests per minute' of service 'sheets.googleapis.com' for consumer 'project_number:522309567947'.", 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_location': 'global', 'quota_limit': 'WriteRequestsPerMinutePerProject', 'quota_limit_value': '900', 'quota_metric': 'sheets.googleapis.com/write_requests', 'service': 'sheets.googleapis.com', 'consumer': 'projects/522309567947'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]}

In [1]:
text = "Files Processed: ['00122-Tal Global Alliances Limited - Supplier PO - Data.csv', '00120-HKS - Supplier PO - Data.csv', '00123-Valerius Textils SA - Supplier PO - Data.csv', '00114-Star Garments - Supplier PO - Data.csv', '00112-Saitex International Dong Nai - Supplier PO - Data.csv', '00121-Tal Global Alliances Limited - Supplier PO - Data.csv', '00119-HKS - Supplier PO - Data.csv', '00115-HKS - Supplier PO - Data.csv', '00117-Star Garments - Supplier PO - Data.csv', '00113-Lamosa Jeans - Supplier PO - Data.csv', '00111-Saitex International Dong Nai - Supplier PO - Data.csv', '00107-Lamosa Jeans - Supplier PO - Data.csv', '00106-Star Garments - Supplier PO - Data.csv', '00103-Saitex International Dong Nai - Supplier PO - Data.csv', '00104-Velcorex - Supplier PO - Data.csv', '00110-Lamosa Jeans - Supplier PO - Data.csv', '00109-HKS - Supplier PO - Data.csv', '00102-Saitex International Dong Nai - Supplier PO - Data.csv', '00105-Marlane - Supplier PO - Data.csv', '00108-Lamosa Jeans - Supplier PO - Data.csv', '00096-Saitex International Dong Nai - Supplier PO - Data.csv', '00091-Spring Near East FZCO - Supplier PO - Data.csv', '00099-Valerius Textils SA - Supplier PO - Data.csv', '00095-Saitex International Dong Nai - Supplier PO - Data.csv', '00101-Saitex International Dong Nai - Supplier PO - Data.csv', '00098-Valerius Textils SA - Supplier PO - Data.csv', '00100-Saitex International Dong Nai - Supplier PO - Data.csv', '00097-Saitex International Dong Nai - Supplier PO - Data.csv', '00092-CBI - Supplier PO - Data.csv', '00094-Saitex International Dong Nai - Supplier PO - Data.csv', '00086-Valerius Textils SA - Supplier PO - Data.csv', '00083-Saitex International Dong Nai - Supplier PO - Data.csv', '00082-Saitex International Dong Nai - Supplier PO - Data.csv', '00087-Valerius Textils SA - Supplier PO - Data.csv', '00088-Spring Near East FZCO - Supplier PO - Data.csv', '00079-Fontoli – Confeções Têxteis, Lda - Supplier PO - Data.csv', '00089-Spring Near East FZCO - Supplier PO - Data.csv', '00081-Tessuti Di Sondrio - Supplier PO - Data.csv', '00090-Spring Near East FZCO - Supplier PO - Data.csv', '00080-Lamosa Jeans - Supplier PO - Data.csv', '00069-Lamosa Jeans - Supplier PO (1) - Data.csv', '00076-Spring Near East FZCO - Supplier PO - Data.csv', '00074-Spring Near East FZCO - Supplier PO - Data.csv', '00072-Lamosa Jeans - Supplier PO - Data.csv', '00073-CBI - Supplier PO - Data.csv', '00070-Lamosa Jeans - Supplier PO - Data.csv', '00078-Fontoli – Confeções Têxteis, Lda - Supplier PO - Data.csv', '00075-Spring Near East FZCO - Supplier PO - Data.csv', '00077-Spring Near East FZCO - Supplier PO - Data.csv', '00071-Lamosa Jeans - Supplier PO - Data.csv', '00061-Spring Near East FZCO - Supplier PO - Data.csv', '00056-Valerius Textils SA - Supplier PO - Data.csv', '00067-Lamosa Jeans - Supplier PO - Data.csv', '00059-Valerius Textils SA - Supplier PO - Data.csv', '00062-Star Garments - Supplier PO - Data.csv', '00068-Lamosa Jeans - Supplier PO - Data.csv', '00060-Saitex International Dong Nai - Supplier PO - Data.csv', '00058-Valerius Textils SA - Supplier PO (1) - Data.csv', '00066-Lamosa Jeans - Supplier PO - Data.csv', '00064-Saitex International Dong Nai - Supplier PO - Data.csv', '00048-Star Garments - Supplier PO - Data.csv', '00049-Star Garments - Supplier PO - Data.csv', '00055-Lamosa Jeans - Supplier PO - Data.csv', '00046-Ritedu - Supplier PO - Data.csv', '00052-Star Garments - Supplier PO - Data.csv', '00050-Star Garments - Supplier PO - Data.csv', '00051-Star Garments - Supplier PO - Data.csv', '00053-Star Garments - Supplier PO - Data.csv', '00054-Panko Pantolon LTD.STI. - Supplier PO - Data.csv', '00047-Star Garments - Supplier PO - Data.csv', '00043-Ritedu - Supplier PO - Data.csv', '00044-Saitex International Dong Nai - Supplier PO Extra Units 8.04.24 - Data.csv', '00044-Saitex International Dong Nai - Supplier PO - Data.csv', '00038-Star Garments - Supplier PO - Data.csv', '00041-Saitex International Dong Nai - Supplier PO - Data.csv', '00039-Saitex International Dong Nai - Supplier PO - Data.csv', '00040-Saitex International Dong Nai - Supplier PO - Data.csv', '00037-Star Garments - Supplier PO - Data.csv', '00042-HKS - Supplier PO - Data.csv', '00045-Saitex International Dong Nai - Supplier PO - Data.csv', '00036-Star Garments - Supplier PO - Data.csv', '00035-Star Garments - Supplier PO Plus Check 21.03.24 - Data.csv']"

text.count(',')

83

In [ ]:
problematic_files.sort()